# Quantum Rectangle Identifier
The code below contains the function:    
    -isRect([int], [int], [int], [int]) that when called will return whether the inputted numbers can define a rectangle or not   
    -The functions needed to run isRect()   
    -Code for testing isRect()

## Imports

In [1]:
import qiskit
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Importing from qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

## Functions needed for isRect()
XOR() applies an xor to qubit [a] and [b] storing result in [output]

In [3]:
#XOR
def XOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)

comp() will generate the quantum circuit needed to compare two numbers stored in qubits as binary

In [4]:
#Compare Nums
def comp(qc, reg0, reg1, regC, outQ, n):
    #Comparing bits in reg0 and reg1 storing result in regC 1 = same 0 = different
    i=0
    while i<n:
        XOR(qc, reg0[i], reg1[i], regC[i])
        qc.x(regC[i])
        i+=1
    #If the two numbers are the same storing 1 in outQ 0 if not
    qc.mct(regC, outQ)
    #Resetting regC
    i=0
    while i<n:
        XOR(qc, reg0[i], reg1[i], regC[i])
        qc.x(regC[i])
        i+=1
    return qc

isEqual() will generate the quantum circuit needed to check if any of 4 numbers equals another of the 4 numbers

In [5]:
#Check Equality
def isEqual(qc, a_qubits, b_qubits, c_qubits, d_qubits, comp_qubits, out_qubits, num, n):
    qc = comp(qc, a_qubits, b_qubits, comp_qubits, out_qubits[num], n)
    qc = comp(qc, a_qubits, c_qubits, comp_qubits, out_qubits[num+1], n)
    qc = comp(qc, a_qubits, d_qubits, comp_qubits, out_qubits[num+2], n)
    return qc

inputBits() will take in a quantum circuit, a quantum register, and a string of 0s and 1s representing a binary number and return the quantum circuit that stores the string in the register

In [6]:
#Fill qubits
def inputBits(qc, reg, string):
    i=0
    for c in string:
        if int(c) == 1:
            qc.x(reg[i])
        i+=1
    return qc

qTemp() is the overall function that generates the quantum circuit for isRect()

In [7]:
#Quantum Function - PreCircuit?
def qTemp(items):
    length = len(items)
    n = len(items[0])
    cbits = ClassicalRegister(12, name='cbits')
    
    #Creating Seperate registers for each variable
    a_qubits = QuantumRegister(n, name='a')
    b_qubits = QuantumRegister(n, name='b')
    c_qubits = QuantumRegister(n, name='c')
    d_qubits = QuantumRegister(n, name='d')
    comp_qubits = QuantumRegister(n, name='comp')
    out_qubits = QuantumRegister(12, name='outs')
    out_qubit = QuantumRegister(1, name='out')
    cbit = ClassicalRegister(1, name='cbit')
    qc = QuantumCircuit(a_qubits, b_qubits, c_qubits, d_qubits, comp_qubits, out_qubits, out_qubit, cbit)
    
    #Inputing values into circuit
    qc = inputBits(qc, a_qubits, items[0])
    qc = inputBits(qc, a_qubits, items[1])
    qc = inputBits(qc, a_qubits, items[2])
    qc = inputBits(qc, a_qubits, items[3])
    
    #Computing
    qc = isEqual(qc, a_qubits, b_qubits, c_qubits, d_qubits, comp_qubits, out_qubits, 0, n)
    qc = isEqual(qc, b_qubits, a_qubits, c_qubits, d_qubits, comp_qubits, out_qubits, 3, n)
    qc = isEqual(qc, c_qubits, b_qubits, a_qubits, d_qubits, comp_qubits, out_qubits, 6, n)
    qc = isEqual(qc, d_qubits, b_qubits, c_qubits, a_qubits, comp_qubits, out_qubits, 9, n)
    
    qc.mct(out_qubits, out_qubit)
    
    #Measuring
    qc.measure(out_qubit, cbit)
    qc.draw(output='mpl', filename='my_circuit.png')
    sim(qc)
    return sim(qc)

sim() will simulate the quantum circuit generated by qTemp() to gain the results

In [8]:
#Circuit Simulator
def sim(qc):
    aer_sim = Aer.get_backend('aer_simulator')
    qobj = assemble(qc)
    result = aer_sim.run(qobj).result()
    counts = result.get_counts()
    return counts

## isRect([int], [int], [int], [int])
isRect() will return a 1 if the four inputted ints can define a rectangle and a 0 if the four inputted ints cannot define a rectangle

In [11]:
#Rectangle finding Function
def isRect(a, b, c, d):
    
    #converting to binary
    a = format(a,'b')
    b = format(b,'b')
    c = format(c,'b')
    d = format(d,'b')
    sides = [a,b,c,d]
    
    #padding to make all the same length
    length = len(a)
    for i in sides:
        if len(i) > length:
            length = len(i)
    i = 0
    while i < 4:
        while len(sides[i]) < length:
            sides[i] = '0'+sides[i]
        i+=1
    
    #Determining if it is a rectangle
    counts = qTemp(sides)
    if '0' in counts:
        return 0
    return 1

## Code for Testing isRect()

In [ ]:
#Testing
i = 10 #number of loops to test
while i > 0:
    print(i)
    #Generating random numbers to test with
    a = random.randint(1,7)
    b = random.randint(1,7)
    c = random.randint(1,7)
    d = random.randint(1,7)
    print(str(a) + "x" + str(b) + "x" + str(c) + "x" + str(d) + ": " + str(isRect(a,b,c,d)))
    print(str(a) + "x" + str(a) + "x" + str(b) + "x" + str(b) + ": " + str(isRect(a,a,b,b)))
    print(str(b) + "x" + str(a) + "x" + str(b) + "x" + str(a) + ": " + str(isRect(a,b,c,d)))
    print(str(a) + "x" + str(b) + "x" + str(b) + "x" + str(a) + ": " + str(isRect(a,a,b,b)))
    print(str(a) + "x" + str(a) + "x" + str(a) + "x" + str(a) + ": " + str(isRect(a,a,b,b)))
    i-=1

10
7x3x3x1: 0
7x7x3x3: 1


## Random Notes

Using Qiskit's example of a Sudoku solver as an example I will make a rectangle identifier.
Need to check if:
a=b||a=c||a=d & b=c||b=d
Where b/c equals the variable a does not.
IDEA: Run Grover's alg twice 
  ~issue only know if there is a pair but not what it pairs with. Maybe run w/ QCounting =4 good =3 bad =2 check next =1 bad
    1. Find A equivalence
        If A has a pair then grover's alg should output 100% since we will only run it once, N<4 might do 50/50 if A matches two?
    2. Find B/C equivalence

## Code unimplemented due to running out of time
This code was going to be used to implement grovers algorithm but due to time constraints I was unable to apply it above

In [ ]:
#Apply Hadamard to every qubit in a given circuit
def applyHadamard(qc, qubits):
    for q in range(qubits):
        qc.h(q)
    return qc

In [ ]:
#Apply x to every qubit in a given circuit
def applyX(qc, qubits):
    for q in range(qubits):
        qc.x(q)
    return qc

In [ ]:
#Apply Z to every qubit in a given circuit
def applyZ(qc, qubits):
    for q in range(qubits):
        qc.z(q)
    return qc

In [ ]:
#Quantum Function
def qFun(items):
    simulator = Aer.get_backend('aer_simulator') #Simulator
    length = len(items) 
    n = len(items[0]) #how many qubits are needed
    n = 3
    #Circuit initialization
    grover = QuantumCircuit(n)
    grover = applyHadamard(grover, n)
    #Applying Oracle
    
    #Applying Diffuser
    grover = applyHadamard(grover, n) #Hadamard
    grover = applyX(grover,n) #X
    grover.h(n-1) #MultZ                   THIS MIGHT NOT BE CORRECT. IF ISSUES DELETE THIS LINE
    grover.mct(list(range(n-1)), n-1) #MultZ
    grover.h(n-1) #MultZ                   THIS MIGHT NOT BE CORRECT. IF ISSUES DELETE THIS LINE
    grover = applyX(grover, n) #X
    grover = applyHadamard(grover, n) #Hadamard
    print("qFUN")
    grover.draw(output='mpl', filename='my_circuit.png')